In [1]:
import pandas as pd
import numpy as np
from sklearn import tree
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
#Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#Import Dataset

dataset = pd.read_excel('/content/drive/My Drive/ML and AI/Assignments/Dataset/Bank Loan Dataset/Bank_Personal_Loan_Modelling.xlsx', sheet_name=1)
dataset.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [7]:
#Drop unnecessary columns

dataset1 = dataset.drop(["ID", "ZIP Code"], axis=1)
dataset2 = dataset1.dropna()
main_dataset = dataset2.drop_duplicates()

In [8]:
#Check for null values

main_dataset.isna().sum()

Age                   0
Experience            0
Income                0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [9]:
# check the columns in the dataset

main_dataset.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [14]:
#Preprocess the data, CCAvg is a float number, round it so that it becomes integer

main_dataset['CCAvg'] = np.round(main_dataset['CCAvg'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
#Identify significant columns by using Random Forest

features = ['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
            'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
            'Online', 'CreditCard']
rf_model = RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)
rf_model.fit(X=main_dataset[features], y=main_dataset['Personal Loan'])


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=2,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [18]:
#Print the OOB accuracy value

print("OOB Accuracy is ", rf_model.oob_score_)

OOB Accuracy is  1.0


In [19]:
#Identify the important features

for feature, imp in zip (features, rf_model.feature_importances_):
  print(feature, imp)

Age 0.011459497188527196
Experience 0.01186349457221733
Income 0.1729510109639513
Family 0.03832431788569649
CCAvg 0.06915505402833791
Education 0.07114157209096983
Mortgage 0.019736867869793902
Personal Loan 0.5644370610237928
Securities Account 0.0019821453274001047
CD Account 0.033133781091257614
Online 0.0023528164026528535
CreditCard 0.0034623815554027143


In [20]:
#Take The Important features from the above output and start building the decision tree
#Important features are Income, CCAvg and Education

#Initialize Decision tree classifier

tree_model = tree.DecisionTreeClassifier(max_depth=8)

In [21]:
#Build Model

predictors = pd.DataFrame([main_dataset['Income'], main_dataset['CCAvg'], main_dataset['Education']]).T
tree_model.fit(X=predictors, y=main_dataset['Personal Loan'])

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=8, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [22]:
#Generate Dot file so that you can visually see the decision tree

with open("Dtree_BankLoan.dot", 'w') as f:
  f = tree.export_graphviz(tree_model, feature_names=['Income','CCAvg','Education'], out_file=f)

In [24]:
#Check your model's score

tree_model.score(X=predictors, y=main_dataset['Personal Loan'])

0.9753358732705033